In [ ]:

from gymu.utils import overload

@overload
def f():
    pass

@f.args(int, int)
def f(x, y):
    print('two integers')

@f.args(float)
def f(x):
    print('one float')

    
print(f.cases)
f(1.)
f(1,1)

In [5]:
import glob
import numpy as np
import gymu
from gymu.data import Composable
import gym_pygame
import jnu as J


env = gymu.make("Alone-v0")
iterator = gymu.iterator(env, mode=gymu.mode.sard, max_length=10)
gymu.data.write_episode(iterator, path="Alone-v0/ep-0.tar.gz")
gymu.data.write_episode(iterator, path="Alone-v0/ep-1.tar.gz")

episodes = glob.glob("./Alone-v0/*.tar.gz")
dataset = gymu.data.dataset(episodes).then(Composable.decode())
#dataset = dataset.then(gymu.data.Composable.mode(gymu.mode.sasd))

for x in dataset:
    #print(x.keys())
    print(x['done'])


#s = [np.concatenate([x.state, x.next_state], axis=-1) for x in dataset]
#J.images(s)              

0it [00:00, ?it/s]

0it [00:00, ?it/s]

False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True


In [ ]:
import jnu as J
import itertools
import gymu
import glob
from gymu.data import Composable


from torch.utils.data import DataLoader



            
            

episodes = glob.glob("./NORMAL-300k/*.tar.gz")

dataset = gymu.data.dataset(episodes)
dataset = dataset.then(Composable.mode(gymu.mode.sa))
dataset = dataset.shuffle(256).batched(256)
loader = DataLoader(dataset, batch_size=None, num_workers=12, prefetch_factor=16)

for (s, a) in loader:
    print(s.shape, a.shape)